<a href="https://colab.research.google.com/github/Marcel0609/Thesis-Code/blob/main/Thesis_Simulation_Fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CELL 1 — Setup + Auth + Connect to Google Sheets

!pip -q install gspread gspread-dataframe

from google.colab import auth
auth.authenticate_user()

import google.auth
import gspread
import pandas as pd
from gspread_dataframe import get_as_dataframe

# Ini buat link spreadsheet
SHEET_URL = "https://docs.google.com/spreadsheets/d/1WBIQ_YBH1R0_fQ8R7mgKPyKS3MxPeoTjGdzHD3J4J3A/edit?usp=sharing"

def connect_by_url(sheet_url: str):
    creds, _ = google.auth.default()
    gc = gspread.authorize(creds)
    return gc.open_by_url(sheet_url)

sh = connect_by_url(SHEET_URL)
print("✅ Connected to:", sh.title)
print("Worksheets:")
for ws in sh.worksheets():
    print(" -", ws.title)


✅ Connected to: Simulation Marcel Sheet
Worksheets:
 - Route_Config
 - Mock_Demand (HBLP)
 - Mock_Demand (HBJM)
 - Results_RouteDay
 - Results_Daily
 - Mock_Demand_Year (HBJM)
 - Mock_Demand_Year (HBLP)
 - Results_Monthly
 - Run_Diagnostics
 - Results_Yearly


In [ ]:
# Code Block 2

import re
import numpy as np
import pandas as pd
from datetime import date, datetime
from zoneinfo import ZoneInfo

import gspread
import google.auth
from gspread_dataframe import get_as_dataframe, set_with_dataframe

TZ = ZoneInfo("Europe/Copenhagen")

MONTH_ID = {1:"Januari",2:"Februari",3:"Maret",4:"April",5:"Mei",6:"Juni",7:"Juli",8:"Agustus",9:"September",10:"Oktober",11:"November",12:"Desember"}
DAY_ID   = {1:"Senin",2:"Selasa",3:"Rabu",4:"Kamis",5:"Jumat",6:"Sabtu",7:"Minggu"}

# Config
MAX_AGING = 3
LAMBDA = 3.0

DEMAND_TABS = {
    "HBJM": "Mock_Demand (HBJM)",
    "HBLP": "Mock_Demand (HBLP)",
}
ROUTE_TAB = "Route_Config"


SCENARIOS_TO_RUN = [
    ("HBJM", "HBJM Baseline"),
    ("HBJM", "HBJM VRP"),
    ("HBLP", "HBLP Baseline"),
    ("HBLP", "HBLP VRP"),
]

# Helpers
def connect_by_url(sheet_url: str):
    creds, _ = google.auth.default()
    gc = gspread.authorize(creds)
    return gc.open_by_url(sheet_url)

def now_cph():
    return datetime.now(TZ).strftime("%Y-%m-%d %H:%M:%S")

def upsert_worksheet(sh, title: str, rows: int = 2000, cols: int = 50):
    try:
        return sh.worksheet(title)
    except Exception:
        return sh.add_worksheet(title=title, rows=rows, cols=cols)

def write_df(sh, sheet_name: str, df: pd.DataFrame, meta: dict | None = None):
    ws = upsert_worksheet(sh, sheet_name, rows=max(2000, len(df)+50), cols=max(50, df.shape[1]+10))
    ws.clear()
    ws.update(values=[["Last updated (Europe/Copenhagen)", now_cph()]], range_name="A1:B1")
    if meta:
        meta_items = [f"{k}={v}" for k, v in meta.items()]
        ws.update(values=[meta_items], range_name="A2")
    set_with_dataframe(ws, df, row=3, col=1, include_index=False)
    print(f"✅ Wrote {len(df)} rows -> '{sheet_name}' (data mulai row=3)")

def norm_addr(x: str) -> str:
    return re.sub(r"\s+", " ", str(x).replace("\u00a0", " ").strip()).upper()

def is_depot(a: str) -> bool:
    a = norm_addr(a)
    return a.startswith("HUB ") or a.startswith("WHS ")

def clean_number(x):
    if x is None:
        return np.nan
    if isinstance(x, (int, float, np.number)) and not (isinstance(x, float) and np.isnan(x)):
        return float(x)

    s = str(x).strip()
    if not s or s.lower() in {"nan","none","-","—"}:
        return np.nan

    s = s.replace("Rp", "").replace("rp", "").replace(" ", "")

    if re.search(r"^\d{1,3}(\.\d{3})+(\,\d+)?$", s):
        s = s.replace(".", "")
        s = s.replace(",", ".")
    elif "," in s and "." in s:
        if s.rfind(",") > s.rfind("."):
            s = s.replace(".", "").replace(",", ".")
        else:
            s = s.replace(",", "")
    elif "," in s and "." not in s:
        if re.search(r",\d{1,2}$", s):
            s = s.replace(",", ".")
        else:
            s = s.replace(",", "")

    try:
        return float(s)
    except Exception:
        return np.nan

def _parse_col_as_date(col):
    if isinstance(col, (date, datetime, pd.Timestamp)):
        return pd.to_datetime(col).date()
    s = str(col).strip()
    if not s:
        return None
    dt = pd.to_datetime(s, errors="coerce", dayfirst=True)
    if pd.isna(dt):
        return None
    return dt.date()

def _drop_empty_and_unnamed_cols(df: pd.DataFrame) -> pd.DataFrame:
    keep = []
    for c in df.columns:
        cs = str(c).replace("\u00a0"," ").strip()
        if not cs or cs.lower() == "nan":
            continue
        if cs.lower().startswith("unnamed"):
            continue
        keep.append(c)
    df = df.loc[:, keep].copy()
    df = df.dropna(axis=1, how="all")
    return df

def fmt_col(d: date) -> str:
    return d.strftime("%d-%b-%y")

def fmt_tanggal_id(d: date) -> str:
    return f"{d.day} {MONTH_ID[d.month]}"

def load_demand_existing(sh, area: str):
    title = DEMAND_TABS[area]
    ws = sh.worksheet(title)

    df = get_as_dataframe(ws, evaluate_formulas=True).dropna(how="all")
    df.columns = df.columns.astype(str).str.strip()
    df = _drop_empty_and_unnamed_cols(df)

    if "Address_ID" not in df.columns:
        if "Address" in df.columns:
            df = df.rename(columns={"Address":"Address_ID"})
        else:
            df = df.rename(columns={df.columns[0]:"Address_ID"})

    df["Address_ID"] = df["Address_ID"].astype(str).map(norm_addr)
    df = df[~df["Address_ID"].isin({"ADDRESS","ADDRESS_ID","NAN",""})].copy()

    total_pat_exact = r"^(TOTAL(\s*DEMAND)?|JUMLAH|GRAND\s*TOTAL|SUM)$"
    df = df[~df["Address_ID"].str.match(total_pat_exact, na=False)].copy()
    df = df[~df["Address_ID"].str.contains(r"^TOTAL\b", regex=True, na=False)].copy()
    df = df.reset_index(drop=True)

    date_to_cols = {}
    for c in df.columns:
        if c == "Address_ID":
            continue
        d = _parse_col_as_date(c)
        if d is None:
            continue
        date_to_cols.setdefault(d, []).append(c)

    if not date_to_cols:
        raise KeyError(f"Demand tab '{title}' tidak ada kolom tanggal yang bisa diparse. Kolom: {list(df.columns)}")

    duplicate_header_count = sum(max(0, len(v)-1) for v in date_to_cols.values())

    agg = pd.DataFrame({"Address_ID": df["Address_ID"]})
    for d in sorted(date_to_cols.keys()):
        cols = date_to_cols[d]
        vals = None
        for c in cols:
            v = df[c].map(clean_number).fillna(0.0)
            vals = v if vals is None else (vals + v)
        agg[fmt_col(d)] = vals.astype(float)

    day_infos = [{"col": fmt_col(d), "date": d} for d in sorted(date_to_cols.keys())]
    keep_cols = ["Address_ID"] + [x["col"] for x in day_infos]
    agg = agg.loc[:, keep_cols].copy()

    return ws.title, agg.reset_index(drop=True), day_infos, duplicate_header_count

def _parse_oper_days(x):
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return []
    s = str(x).strip()
    if not s:
        return []
    tokens = [t.strip() for t in re.split(r"[,\s;]+", s) if t.strip()]
    day_map = {"senin":1,"selasa":2,"rabu":3,"kamis":4,"jumat":5,"sabtu":6,"minggu":7,
               "mon":1,"tue":2,"wed":3,"thu":4,"fri":5,"sat":6,"sun":7}
    out = []
    for t in tokens:
        tl = t.lower()
        if tl in day_map:
            out.append(day_map[tl]); continue
        try:
            v = int(float(t))
            if 0 <= v <= 6: v += 1
            out.append(v)
        except:
            pass
    return [d for d in sorted(set(out)) if 1 <= d <= 7]

def load_route_config(sh):
    ws = sh.worksheet(ROUTE_TAB)
    df = get_as_dataframe(ws).dropna(how="all")
    df.columns = df.columns.astype(str).str.strip()
    df = df.rename(columns={
        "Route_Name":"Route",
        "Distance_km":"Distance",
        "VarCost_per_km":"Var_Cost",
        "Total_Route_Cost":"Total_Cost",
    })
    if "Scenario" not in df.columns:
        raise KeyError(f"Route_Config tidak punya kolom 'Scenario'. Kolom: {list(df.columns)}")
    df["Scenario"] = df["Scenario"].astype(str).str.strip()

    # include Qty in Month
    for c in ["Capacity","Fixed_Cost","Distance","Var_Cost","Total_Cost","Qty in Month"]:
        if c in df.columns:
            df[c] = df[c].map(clean_number)

    return ws.title, df

def parse_routes_for_scenario(route_df_all: pd.DataFrame, scenario: str, lambd: float):
    df = route_df_all[route_df_all["Scenario"].str.casefold() == str(scenario).strip().casefold()].reset_index(drop=True)
    if df.empty:
        available = sorted([x for x in route_df_all["Scenario"].unique().tolist() if x and str(x).lower() != "nan"])
        raise ValueError(f"Scenario '{scenario}' tidak ada. Tersedia: {available}")

    required = ["Addresses","Oper_Days","Capacity","Fixed_Cost"]
    miss = [c for c in required if c not in df.columns]
    if miss:
        raise KeyError(f"Scenario '{scenario}' missing columns={miss}. Kolom ada: {list(df.columns)}")

    def _colnorm(s): return re.sub(r"[^a-z0-9]+", "", str(s).strip().lower())
    qty_col = None
    for c in df.columns:
        if _colnorm(c) == _colnorm("Qty in Month"):
            qty_col = c
            break

    cap_med   = np.nanmedian(df["Capacity"].values) if "Capacity" in df.columns else np.nan
    fixed_med = np.nanmedian(df["Fixed_Cost"].values) if "Fixed_Cost" in df.columns else np.nan
    total_med = np.nanmedian(df["Total_Cost"].values) if "Total_Cost" in df.columns else np.nan
    dist_med  = np.nanmedian(df["Distance"].values) if "Distance" in df.columns else np.nan
    var_med   = np.nanmedian(df["Var_Cost"].values) if "Var_Cost" in df.columns else np.nan

    candidates = []
    addr_candidates = {}
    penalty_zero_routes = 0

    for _, r in df.iterrows():
        route_label = str(r["Addresses"]).strip()

        addrs_raw = [a.strip() for a in route_label.split("-") if a.strip()]
        addrs_norm = [norm_addr(a) for a in addrs_raw]

        seen = set()
        service_addrs = []
        for a in addrs_norm:
            if is_depot(a):
                continue
            if a in seen:
                continue
            seen.add(a)
            service_addrs.append(a)

        oper_days = _parse_oper_days(r.get("Oper_Days"))
        if not oper_days:
            oper_days = [1,2,3,4,5,6,7]

        cap = clean_number(r.get("Capacity"))
        if not np.isfinite(cap) or cap <= 0:
            cap = cap_med if np.isfinite(cap_med) and cap_med > 0 else 1.0

        fixed = clean_number(r.get("Fixed_Cost"))
        if not np.isfinite(fixed):
            fixed = fixed_med if np.isfinite(fixed_med) else 0.0

        total_cost = clean_number(r.get("Total_Cost"))
        if not np.isfinite(total_cost):
            total_cost = total_med if np.isfinite(total_med) else np.nan

        dist = clean_number(r.get("Distance"))
        var_km = clean_number(r.get("Var_Cost"))

        if np.isfinite(total_cost):
            regular_cost = float(total_cost)
        else:
            if not np.isfinite(dist):
                dist = dist_med if np.isfinite(dist_med) else 0.0
            if not np.isfinite(var_km):
                var_km = var_med if np.isfinite(var_med) else 0.0
            regular_cost = float(fixed) + float(dist) * float(var_km)

        var_part = np.nan
        if np.isfinite(dist) and np.isfinite(var_km):
            var_part = float(dist) * float(var_km)
        elif np.isfinite(total_cost) and np.isfinite(fixed):
            var_part = max(float(total_cost) - float(fixed), 0.0)

        if np.isfinite(var_part) and cap > 0:
            penalty_per_unit = float(lambd) * (float(var_part) / float(cap))
        else:
            penalty_per_unit = 0.0
            penalty_zero_routes += 1

        qty_in_month = clean_number(r.get(qty_col)) if qty_col is not None else np.nan

        idx = len(candidates)
        candidates.append({
            "route_label": route_label,
            "service_addrs_raw": service_addrs,
            "oper_days": oper_days,
            "capacity": float(cap),
            "regular_cost": float(regular_cost) if np.isfinite(regular_cost) else 0.0,
            "penalty_per_unit": float(penalty_per_unit),
            "vehicle": r.get("Vehicle",""),
            "qty_in_month": qty_in_month,
        })

        for a in service_addrs:
            addr_candidates.setdefault(a, []).append(idx)

    def score(idx: int):
        c = candidates[idx]
        return (c["regular_cost"], -len(c["oper_days"]), -c["capacity"], idx)

    best_route_for_addr = {a: min(lst, key=score) for a, lst in addr_candidates.items()}

    routes = []
    addr_to_route = {}

    for idx, c in enumerate(candidates):
        assigned = [a for a in c["service_addrs_raw"] if best_route_for_addr.get(a) == idx]
        for a in assigned:
            addr_to_route[a] = c["route_label"]

        routes.append({
            "route_label": c["route_label"],
            "service_addrs": assigned,
            "oper_days": c["oper_days"],
            "capacity": c["capacity"],
            "regular_cost": c["regular_cost"],
            "penalty_per_unit": c["penalty_per_unit"],
            "vehicle": c["vehicle"],
            "qty_in_month": c["qty_in_month"],
        })

    penalty_zero_share = (penalty_zero_routes / len(candidates)) if candidates else np.nan
    return routes, addr_to_route, penalty_zero_routes, penalty_zero_share

def run_sim_engine(demand_df, day_infos, routes, addr_to_route, scenario_label, max_aging):
    served = set(addr_to_route.keys())
    keep_mask = demand_df["Address_ID"].isin(served)
    dropped = sorted(set(demand_df.loc[~keep_mask, "Address_ID"].tolist()))
    demand_df = demand_df.loc[keep_mask].copy().reset_index(drop=True)

    dests = demand_df["Address_ID"].unique().tolist()
    backlog = {a: {age: 0.0 for age in range(max_aging + 1)} for a in dests}

    addr_penalty = {}
    for r in routes:
        ppu = float(r["penalty_per_unit"])
        for a in r.get("service_addrs", []):
            if a in served:
                addr_penalty[a] = ppu

    demand_records = demand_df.to_dict("records")

    daily_rows = []
    route_day_rows = []
    monthly = {}
    yearly = {"ship": 0.0, "reg": 0.0, "pen": 0.0, "tot": 0.0, "emg": 0.0}

    for info in day_infos:
        col = info["col"]
        dt = info["date"]
        iso_dow = dt.isoweekday()
        ym = (dt.year, dt.month)

        if ym not in monthly:
            monthly[ym] = {"ship": 0.0, "reg": 0.0, "pen": 0.0, "tot": 0.0, "emg": 0.0}

        tanggal = fmt_tanggal_id(dt)
        hari = DAY_ID[iso_dow]

        for a in backlog.keys():
            for age in range(max_aging, 0, -1):
                backlog[a][age] = backlog[a][age - 1]
            backlog[a][0] = 0.0

        for row in demand_records:
            a = row.get("Address_ID", "")
            if a not in backlog:
                continue
            q = clean_number(row.get(col, 0))
            q = 0.0 if (not np.isfinite(q)) else float(q)
            if q < 0:
                q = 0.0
            backlog[a][0] += q

        shipped_today = 0.0
        regular_cost_today = 0.0
        planned_capacity_today = 0.0
        used_capacity_today = 0.0
        routes_operated = 0

        for r in routes:
            scheduled = iso_dow in r["oper_days"]
            service_addrs = r.get("service_addrs", [])
            cap = float(r["capacity"])

            operated = bool(scheduled and len(service_addrs) > 0)
            if operated:
                planned_capacity_today += cap

            agg = 0.0
            for a in service_addrs:
                if a in backlog:
                    agg += sum(backlog[a].values())

            shipped = 0.0
            cap_left = cap

            if operated and cap_left > 0:
                for age in range(max_aging, -1, -1):
                    if cap_left <= 0:
                        break
                    for a in service_addrs:
                        if cap_left <= 0:
                            break
                        if a not in backlog:
                            continue
                        q = backlog[a][age]
                        if q <= 0:
                            continue
                        take = min(q, cap_left)
                        backlog[a][age] -= take
                        cap_left -= take
                        shipped += take

                used = cap - cap_left
                used_capacity_today += used
                shipped_today += shipped
                routes_operated += 1

                rcost = float(r["regular_cost"])
                regular_cost_today += rcost
            else:
                used = 0.0
                rcost = 0.0

            route_day_rows.append({
                "Date": dt, "Tanggal": tanggal, "Hari": hari,
                "Scenario": scenario_label,
                "Route": r["route_label"],
                "Operated": bool(operated),
                "Aggregated_Demand_Pre": agg,
                "Capacity": cap,
                "Shipped": shipped,
                "Utilization": (used / cap) if (operated and cap > 0) else np.nan,
                "Regular_Route_Cost": rcost,
                "Penalty_per_Unit": float(r["penalty_per_unit"]),
                "Vehicle": r.get("vehicle", "")
            })

        emergency_units = 0.0
        penalty_cost_today = 0.0
        for a in list(backlog.keys()):
            q = backlog[a][max_aging]
            if q > 0:
                emergency_units += q
                penalty_cost_today += q * float(addr_penalty.get(a, 0.0))
                backlog[a][max_aging] = 0.0

        utilization_today = (used_capacity_today / planned_capacity_today) if planned_capacity_today > 0 else np.nan
        violation_today = 1 if emergency_units > 0 else 0
        total_cost_today = regular_cost_today + penalty_cost_today

        daily_rows.append({
            "Date": dt, "Tanggal": tanggal, "Hari": hari,
            "Scenario": scenario_label,
            "Shipped": shipped_today,
            "Emergency_Units": emergency_units,
            "Violation_Day": violation_today,
            "Routes_Operated": routes_operated,
            "Planned_Capacity": planned_capacity_today,
            "Used_Capacity": used_capacity_today,
            "Utilization": utilization_today,
            "Regular_Cost": regular_cost_today,
            "Penalty_Cost": penalty_cost_today,
            "Total_Cost": total_cost_today
        })

        monthly[ym]["ship"] += shipped_today
        monthly[ym]["emg"] += emergency_units
        monthly[ym]["reg"] += regular_cost_today
        monthly[ym]["pen"] += penalty_cost_today
        monthly[ym]["tot"] += total_cost_today

        yearly["ship"] += shipped_today
        yearly["emg"] += emergency_units
        yearly["reg"] += regular_cost_today
        yearly["pen"] += penalty_cost_today
        yearly["tot"] += total_cost_today

    daily_df = pd.DataFrame(daily_rows)
    route_day_df = pd.DataFrame(route_day_rows)

    violation_days = int(daily_df["Violation_Day"].sum()) if not daily_df.empty else 0
    penalty_share = (yearly["pen"] / yearly["tot"]) if yearly["tot"] > 0 else np.nan

    yearly_row = {
        "Scenario": scenario_label,
        "Days_Simulated": len(daily_df),
        "Date_Start": day_infos[0]["date"] if day_infos else None,
        "Date_End": day_infos[-1]["date"] if day_infos else None,
        "Total_Shipped": yearly["ship"],
        "Total_Emergency_Units": yearly["emg"],
        "Regular_Cost": yearly["reg"],
        "Penalty_Cost": yearly["pen"],
        "Total_Cost": yearly["tot"],
        "Penalty_Share": penalty_share,
        "Violation_Days": violation_days,
        "Violation_Frequency": (violation_days / len(daily_df)) if len(daily_df) > 0 else np.nan,
        "Avg_Utilization": daily_df["Utilization"].mean() if "Utilization" in daily_df.columns else np.nan,
        "Dropped_Demand_Addresses": len(dropped),
    }

    monthly_rows = []
    for (yy, mm), v in sorted(monthly.items()):
        tot = v["tot"]
        monthly_rows.append({
            "Scenario": scenario_label,
            "Year": yy,
            "MonthNum": mm,
            "Month": MONTH_ID[mm],
            "Shipped": v["ship"],
            "Emergency_Units": v["emg"],
            "Regular_Cost": v["reg"],
            "Penalty_Cost": v["pen"],
            "Total_Cost": v["tot"],
            "Penalty_Share": (v["pen"] / tot) if tot > 0 else np.nan
        })
    monthly_df = pd.DataFrame(monthly_rows)

    return pd.DataFrame([yearly_row]), monthly_df, daily_df, route_day_df, dropped


# =========================
# RUN + EXPORT (Run_Diagnostics normal table)
# Frekuensi Pengiriman = Route_Config["Qty in Month"]
# =========================
sh = connect_by_url(SHEET_URL)

route_tab_title, route_df_all = load_route_config(sh)
available_scen = sorted([x for x in route_df_all["Scenario"].unique().tolist() if x and str(x).lower() != "nan"])

all_yearly, all_monthly, all_daily, all_routeday = [], [], [], []
diag_rows = []

def _oper_days_name(oper_days):
    try:
        return ", ".join([DAY_ID[int(d)] for d in oper_days if int(d) in DAY_ID])
    except Exception:
        return ""

for area, scen in SCENARIOS_TO_RUN:
    if scen.casefold() not in [s.casefold() for s in available_scen]:
        diag_rows.append({
            "Area": area, "Scenario": scen, "Status": "SKIPPED",
            "Reason": "Scenario not found in Route_Config",
            "DemandTab": None, "RouteTab": route_tab_title,
            "Days": None, "DateStart": None, "DateEnd": None,
            "DuplicateDateHeadersMerged": None,
            "Routes_Total": None, "Routes_NoStops": None, "Routes_Jalan": None,
            "ServedDestinations": None, "DroppedDemandDestinations": None,
            "PenaltyZeroRoutes": None, "PenaltyZeroShare": None,
            "TotalCost": None, "RegularCost": None, "PenaltyCost": None, "EmergencyUnits": None,
            "Route": None, "Oper_Days_Num": None, "Oper_Days_Name": None,
            "Assigned_Stops": None,
            "Regular_Cost_per_Run": None, "Penalty_per_Unit": None,
            "Frekuensi Pengiriman": None,
        })
        continue

    demand_tab_title, demand_df, day_infos, dup_count = load_demand_existing(sh, area)
    routes, addr_to_route, penalty_zero_routes, penalty_zero_share = parse_routes_for_scenario(route_df_all, scen, LAMBDA)

    ydf, mdf, ddf, rdf, dropped = run_sim_engine(demand_df, day_infos, routes, addr_to_route, scen, MAX_AGING)

    ydf.insert(1, "Area", area); ydf.insert(2, "DemandTab", demand_tab_title); ydf.insert(3, "RouteTab", route_tab_title)
    mdf.insert(1, "Area", area); mdf.insert(2, "DemandTab", demand_tab_title)
    ddf.insert(1, "Area", area); ddf.insert(2, "DemandTab", demand_tab_title)
    rdf.insert(1, "Area", area); rdf.insert(2, "DemandTab", demand_tab_title)

    all_yearly.append(ydf)
    all_monthly.append(mdf)
    all_daily.append(ddf)
    all_routeday.append(rdf)

    horizon_dows = {x["date"].isoweekday() for x in day_infos}

    routes_no_stops = 0
    routes_operable = []
    for r in routes:
        if not r.get("service_addrs"):
            routes_no_stops += 1
            continue
        if set(r["oper_days"]) & horizon_dows:
            routes_operable.append(r)

    scen_row_common = {
        "Area": area,
        "Scenario": scen,
        "Status": "OK",
        "Reason": None,
        "DemandTab": demand_tab_title,
        "RouteTab": route_tab_title,
        "Days": len(day_infos),
        "DateStart": day_infos[0]["date"],
        "DateEnd": day_infos[-1]["date"],
        "DuplicateDateHeadersMerged": dup_count,
        "Routes_Total": len(routes),
        "Routes_NoStops": routes_no_stops,
        "Routes_Jalan": len(routes_operable),
        "ServedDestinations": len(addr_to_route),
        "DroppedDemandDestinations": len(dropped),
        "PenaltyZeroRoutes": penalty_zero_routes,
        "PenaltyZeroShare": penalty_zero_share,
        "TotalCost": float(ydf["Total_Cost"].iloc[0]),
        "RegularCost": float(ydf["Regular_Cost"].iloc[0]),
        "PenaltyCost": float(ydf["Penalty_Cost"].iloc[0]),
        "EmergencyUnits": float(ydf["Total_Emergency_Units"].iloc[0]),
    }

    if len(routes_operable) == 0:
        diag_rows.append({
            **scen_row_common,
            "Route": None,
            "Oper_Days_Num": None,
            "Oper_Days_Name": None,
            "Assigned_Stops": 0,
            "Regular_Cost_per_Run": None,
            "Penalty_per_Unit": None,
            "Frekuensi Pengiriman": None,
        })
    else:
        for r in routes_operable:
            diag_rows.append({
                **scen_row_common,
                "Route": r["route_label"],
                "Oper_Days_Num": ",".join(str(x) for x in r["oper_days"]),
                "Oper_Days_Name": _oper_days_name(r["oper_days"]),
                "Assigned_Stops": len(r.get("service_addrs", [])),
                "Regular_Cost_per_Run": r.get("regular_cost", np.nan),
                "Penalty_per_Unit": r.get("penalty_per_unit", np.nan),
                "Frekuensi Pengiriman": r.get("qty_in_month", np.nan),
            })

yearly_out   = pd.concat(all_yearly, ignore_index=True) if all_yearly else pd.DataFrame()
monthly_out  = pd.concat(all_monthly, ignore_index=True) if all_monthly else pd.DataFrame()
daily_out    = pd.concat(all_daily, ignore_index=True) if all_daily else pd.DataFrame()
routeday_out = pd.concat(all_routeday, ignore_index=True) if all_routeday else pd.DataFrame()
diag_out     = pd.DataFrame(diag_rows)

# order columns nicely
base_cols = [
    "Area","Scenario","Status","Reason","DemandTab","RouteTab",
    "Days","DateStart","DateEnd","DuplicateDateHeadersMerged",
    "Routes_Total","Routes_NoStops","Routes_Jalan",
    "ServedDestinations","DroppedDemandDestinations",
    "PenaltyZeroRoutes","PenaltyZeroShare",
    "TotalCost","RegularCost","PenaltyCost","EmergencyUnits",
    "Route","Oper_Days_Num","Oper_Days_Name","Assigned_Stops",
    "Frekuensi Pengiriman",
    "Regular_Cost_per_Run","Penalty_per_Unit"
]
desired_cols = [c for c in base_cols if c in diag_out.columns] + [c for c in diag_out.columns if c not in base_cols]
diag_out = diag_out.loc[:, desired_cols]

meta = {
    "MAX_AGING": MAX_AGING,
    "LAMBDA": LAMBDA,
    "DropTotalRow": "YES",
    "UniqueStops": "YES (dedup, remove HUB/WHS)",
    "UniqueAssignAcrossRoutes": "YES (best by cost,freq,cap,first)",
    "RouteTab": route_tab_title,
    "ScenariosRequested": " | ".join([s for _, s in SCENARIOS_TO_RUN]),
    "FrekuensiPengirimanSource": "Route_Config['Qty in Month']",
}

write_df(sh, "Results_Yearly", yearly_out, meta=meta)
write_df(sh, "Results_Monthly", monthly_out, meta=meta)
write_df(sh, "Results_Daily", daily_out, meta=meta)
write_df(sh, "Results_RouteDay", routeday_out, meta=meta)
write_df(sh, "Run_Diagnostics", diag_out, meta=meta)

print("\nDONE. Results_* updated (overwrite). Run_Diagnostics: kolom 'Frekuensi Pengiriman' dari Route_Config['Qty in Month'].")


/tmp/ipython-input-2445064569.py:180: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  agg[fmt_col(d)] = vals.astype(float)
/tmp/ipython-input-2445064569.py:180: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  agg[fmt_col(d)] = vals.astype(float)
/tmp/ipython-input-2445064569.py:180: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `ne

✅ Wrote 4 rows -> 'Results_Yearly' (data mulai row=3)
✅ Wrote 20 rows -> 'Results_Monthly' (data mulai row=3)
✅ Wrote 612 rows -> 'Results_Daily' (data mulai row=3)
✅ Wrote 4131 rows -> 'Results_RouteDay' (data mulai row=3)
✅ Wrote 22 rows -> 'Run_Diagnostics' (data mulai row=3)

DONE. Results_* updated (overwrite). Run_Diagnostics: kolom 'Frekuensi Pengiriman' dari Route_Config['Qty in Month'].
